# Looking at pre-dispatch demand forecast errors in 2021

In this example, we will take a look at (30-minute) pre-dispatch ({term}`PREDISPATCH`) demand forecast "error" (the difference between actual and forecasted demand) for 2021. Unlike 5PMD, pre-dispatch extends out to 39 hours ahead, so it's a good dataset to use to look at day-ahead forecast errors.

```{note}
The code below uses functionalities offered by `NEMOSIS` and `NEMSEER`. Alternative approaches (with respect to caching data, data handling, etc.) may be faster than the approach below. 

To speed up computation of forecast error using `NEMOSIS` and `NEMSEER`, we use Python's `multiprocessing` module to dispatch error calculation across multiple Python processes.
```

## Library Imports

In [1]:
# standard libraries
import logging
import multiprocessing as mp
from datetime import datetime, timedelta
from pathlib import Path

# NEM data libraries
# NEMOSIS for actual demand data
# NEMSEER for forecast demand data
import nemosis
from nemseer import compile_data, download_raw_data, generate_runtimes

# data wrangling libraries
import numpy as np
import pandas as pd

# interactive plotting
import plotly.express as px
import plotly.io as pio

# progress bar for error computation
from tqdm.autonotebook import tqdm

# silence NEMSEER and NEMOSIS logging
logging.getLogger("nemseer").setLevel(logging.ERROR)
logging.getLogger("nemosis").setLevel(logging.WARNING)

## Defining our analysis start and end dates

In [2]:
analysis_start = "2021/01/01 00:00:00"
analysis_end = "2022/01/01 00:00:00"

## Obtaining actual demand data from `NEMOSIS`

We will download `DISPATCHREGIONSUM` to access the `TOTALDEMAND` field.

We'll first download the data we need and cache it so that it's ready for computation.

In [3]:
nemosis_cache = Path("nemosis_cache/")
if not nemosis_cache.exists():
    nemosis_cache.mkdir()

In [4]:
nemosis.cache_compiler(
    analysis_start, analysis_end, "DISPATCHREGIONSUM", nemosis_cache, fformat="parquet"
)

## Obtaining forecast demand data from `NEMSEER`

We will download `REGIONSUM` to access the `TOTALDEMAND` field in `PREDISPATCH` forecasts.

We'll first download the data we need and cache it so that it's ready for computation.

In [5]:
download_raw_data(
    "PREDISPATCH",
    "REGIONSUM",
    "nemseer_cache/",
    forecasted_start=analysis_start,
    forecasted_end=analysis_end,
)

In [6]:
def calculate_predispatch_demand_forecast_error(forecasted_time):
    """
    Calculates forecast error (Actual - Forecast) for all PD forecasts for a given forecasted_time.
    
    Args:
        forecasted_time: Datetime string in the form YYYY/mm/dd HH:MM:SS
    Returns:
        pandas DataFrame with forecast error in `TOTALDEMAND` columns, the ahead time
        of the forecast run in `ahead_time`, and the forecasted time in
        `forecasted_time`.
    """
    time = str(forecasted_time).replace("-", "/")
    run_start, run_end = generate_runtimes(time, time, "PREDISPATCH")
    nemseer_data = compile_data(
        run_start,
        run_end,
        time,
        time,
        "PREDISPATCH",
        "REGIONSUM",
        "nemseer_cache/",
        data_format="xr",
    )
    demand_forecasts = nemseer_data["REGIONSUM"]["TOTALDEMAND"]
    nemosis_start = (
        datetime.strptime(time, "%Y/%m/%d %H:%M:%S") - timedelta(minutes=5)
    ).strftime("%Y/%m/%d %H:%M:%S")
    nemosis_data = (
        nemosis.dynamic_data_compiler(
            nemosis_start,
            time,
            "DISPATCHREGIONSUM",
            nemosis_cache,
            filter_cols=["INTERVENTION"],
            filter_values=([0],),
            fformat="parquet",
        )
        .set_index("SETTLEMENTDATE")
        .sort_index()
    )
    regions = ("QLD1", "NSW1", "VIC1", "TAS1", "SA1")
    errors = []
    for region in regions:
        actual_demand = nemosis_data.query("REGIONID==@region")["TOTALDEMAND"][time]
        query_forecasts = demand_forecasts.sel(forecasted_time=time, REGIONID=region)
        error = (actual_demand - query_forecasts).to_dataframe()
        error["ahead_time"] = error["forecasted_time"] - error.index
        error = error.set_index("forecasted_time")
        errors.append(error)
    return pd.concat(errors, axis=0)

To speed up computation, we will use Python’s [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) module. In this example, we use 10 simultaneous processes to parallelise processing, and then concatenate results into a single DataFrame.

tqdm provides us with a progress bar that shows us how many iterations are being completed in a second, as well as the progress over all intervals in the year or interest.

In [7]:
times = pd.date_range(analysis_start, analysis_end, freq="30T")
with mp.Pool(10) as p:
    results = list(
       tqdm(
           p.imap(calculate_predispatch_demand_forecast_error, times), total=len(times)
       )
    )
forecast_error = pd.concat(results, axis=0)

  0%|          | 0/17521 [00:00<?, ?it/s]

## Region-by-region error percentiles

Below we plot regional error percentiles for all ahead times.

In [9]:
region_ahead_percentiles = {}
for region in (regions := ("QLD1", "NSW1", "VIC1", "SA1", "TAS1")):
    quantile_data = []
    region_error = forecast_error.query("REGIONID==@region")
    for quantile in (0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99):
        quantile_result = region_error.groupby(
            region_error["ahead_time"].dt.total_seconds() / (60**2)
        )["TOTALDEMAND"].quantile(quantile)
        quantile_result = pd.concat(
            [
                quantile_result,
                pd.Series(
                    np.repeat(quantile, len(quantile_result)),
                    index=quantile_result.index,
                    name="quantile",
                ),
            ],
            axis=1,
        )
        quantile_data.append(quantile_result)
    quantile_df = pd.concat(quantile_data, axis=0).reset_index()
    region_ahead_percentiles[region] = quantile_df

In [13]:
for region in regions:
    fig = px.line(
        region_ahead_percentiles[region],
        x="ahead_time",
        y="TOTALDEMAND",
        color="quantile",
        title=f"PD {region} Demand Forecast Error (Actual - Forecast)",
        labels={
            "TOTALDEMAND": "Demand Forecast Error (MW)",
            "ahead_time": "Forecast Ahead Time (Hours, Nominal Run Time)",
        },
    )
    fig["layout"]["xaxis"]["autorange"] = "reversed"
    pio.write_html(fig, f"../_static/pd_error_{region}_2021_aheadtime_percentile.html")

```{raw} html
---
file: ../_static/pd_error_NSW1_2021_aheadtime_percentile.html
---
```

```{raw} html
---
file: ../_static/pd_error_VIC1_2021_aheadtime_percentile.html
---
```

```{raw} html
---
file: ../_static/pd_error_QLD1_2021_aheadtime_percentile.html
---
```

```{raw} html
---
file: ../_static/pd_error_SA1_2021_aheadtime_percentile.html
---
```

```{raw} html
---
file: ../_static/pd_error_TAS1_2021_aheadtime_percentile.html
---
```

### Why does the error drop off beyond ~24 hours?

A limited number of periods during the day are actually forecasted beyond 24 hours out.

`PREDISPATCH` is run until the end of the trading day for which bid price band submission has closed (1230 EST). So this means, for example:
- The 1300 PD run will forecast out til 4AM two days away (39 hours)
- But the 1400 PD run will still only forecast out til 4AM two days away (38 hours)
- And the 0800 PD run the next day will still only forecast out til 4AM the next day (20 hours)

So because of this, the number of error samples drops off beyond 15.5 hours ahead (see figure below).

In addition, the runs closer to ~35 hours will be forecasts for periods in the early hours of the morning. These periods tend to have more predictable demand.

In [16]:
sample_count = px.line(
    forecast_error.groupby(forecast_error["ahead_time"].dt.total_seconds() / (60**2))[
        "TOTALDEMAND"
    ]
    .count()
    .rename("Computed Errors"),
    labels={"value": "Count of Samples"},
)
pio.write_html(sample_count, f"../_static/pd_error_2021_ahead_samples.html")

```{raw} html
---
file: ../_static/pd_error_2021_ahead_samples.html
---
```

## NEM-wide Demand Forecast Error, less than 24 hours

Because of the reasons above, we'll focus on ahead times of up to 24 hours.

In [17]:
nem_error = (
    forecast_error.reset_index()
    .groupby(["forecasted_time", "ahead_time"])["TOTALDEMAND"]
    .sum()
    .reset_index()
)
nem_quantile_data = []
for quantile in (0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99):
    nem_quantile_result = nem_error.groupby(
        nem_error["ahead_time"].dt.total_seconds() / (60**2)
    )["TOTALDEMAND"].quantile(quantile)
    nem_quantile_result = pd.concat(
        [
            nem_quantile_result,
            pd.Series(
                np.repeat(quantile, len(nem_quantile_result)),
                index=nem_quantile_result.index,
                name="quantile",
            ),
        ],
        axis=1,
    )
    nem_quantile_data.append(nem_quantile_result)
nem_quantile_df = pd.concat(nem_quantile_data, axis=0).reset_index()

In [19]:
nemwide = px.line(
    nem_quantile_df.query("ahead_time < 24"),
    x="ahead_time",
    y="TOTALDEMAND",
    color="quantile",
    title=f"PD NEM-wide Demand Forecast Error (Actual - Forecast)",
    labels={
        "TOTALDEMAND": "Demand Forecast Error (MW)",
        "ahead_time": "Forecast Ahead Time (Hours, Nominal Run Time)",
    },
)
nemwide["layout"]["xaxis"]["autorange"] = "reversed"
pio.write_html(nemwide, "../_static/pd_error_NEM_2021_ahead_time_percentile.html")

```{raw} html
---
file: ../_static/pd_error_NEM_2021_ahead_time_percentile.html
---
```

## Distributions of Day-Ahead Demand Forecast Error by Region

We can see that the TOTALDEMAND day-ahead demand forecast error distribution is long-tailed for every region.

In [21]:
day_ahead = forecast_error[
    forecast_error["ahead_time"].dt.total_seconds() / (60**2) == 24
]
da_dists = px.histogram(
    day_ahead,
    x="TOTALDEMAND",
    facet_row="REGIONID",
    title="PREDISPATCH Demand Forecast Error 2021 - Day-Ahead (24 hours ahead)",
)
pio.write_html(da_dists, "../_static/pd_error_2021_da_dists.html")

```{raw} html
---
file: ../_static/pd_error_2021_da_dists.html
---
```